## Evaluation of the Trained Model for the Synthetic Public Data

@author M. Schultheiß, T. Dorosti

In [ ]:
import os
import numpy as np
import h5py
from scipy import ndimage
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from scipy.stats import pearsonr

from pyDeleClasses import Slice2D, Slice2DSet
from configuration_paper import cparam, CURRENT_CONFIG, PATH_SERVER_CACHE, POSITION
from functions import mse, made, get_metric, shapiro_wilk, dAgostino_pearson, anderson_darling, plotter, process_PE, getSyntheticData

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=''

## 1. Load data:

### 1a) Load Luna16 data

In [ ]:
metadataPath = '???' # path to metadata .csv file
dataset_name = 'luna16'
test_split_luna = pd.read_csv('./splits/lungvolume_{}_test.csv'.format(dataset_name), header=None)
radiograph_range = range(4, 5) # get only central projections for luna test set
slices_luna = getSyntheticData(metadataPath, PATH_SERVER_CACHE, POSITION, dataset_name, CURRENT_CONFIG, radiograph_range)
_, _, testx_luna, testy_luna, _, _  = slices_luna.split_data_by_csv(test_split_luna)
print("Test Len", len(testx_luna))

In [ ]:
# save pixel spacing info for the test data for total volume calculation
pixel_spacings_luna = []
df_luna = pd.read_csv(metadataPath)

for i in test_split_luna[0]:
    for j in range(len(df_luna)):
        if i == df_luna['identifier'][j]:
            voxel_size_z = df_luna['spacing_0'][j]
            pixel_spacings_luna.append([voxel_size_z])

### 1b) Load PE data

In [ ]:
metadataPath = '???' # path to metadata .csv file
dataset_name = 'PE'
test_split_PE = pd.read_csv('./splits/lungvolume_{}_test.csv'.format(dataset_name), header=None)
radiograph_range = range(4, 5) # get only central projections for PE test set
slices_PE = getSyntheticData(metadataPath, PATH_SERVER_CACHE, POSITION, dataset_name, CURRENT_CONFIG, radiograph_range)
_, _, testx_PE, testy_PE, _, _  = slices_PE.split_data_by_csv(test_split_PE)
print("Test Len", len(testx_PE))

In [ ]:
# get scale factor and pixel size lists for the test data:
pixel_spacings_PE = []
scale_factor_pre_PE =[]
slices_z = []
for i in test_split_PE[0]:
    for j in range(len(df_PE)):
        if int(i[:-5]) == int(df_PE['identifier'][j]):
            voxel_size_z = df_PE['spacing_0'][j]
            num_slices_z = len(df_PE[num_slices'][j].split(','))
            voxel_size_x = float(df_PE['spacing_1'][j].split(',')[0][1:]) 
            voxel_size_y = float(df_PE['spacing_2'][j].split(',')[1][:-1])
            num_slices_x = 512
            z_total_mm = voxel_size_z * num_slices_z
            x_total_mm = voxel_size_x * num_slices_x
            pixel_spacings_PE.append([voxel_size_x, voxel_size_y, voxel_size_z])
            slices_z.append(num_slices_z)
            scale_factor_pre_PE.append(np.round(x_total_mm/z_total_mm, 3))
        

## 2. Load Model and predict:

In [ ]:
model = load_model('model_final.h5')

### 2a) Predict on Luna test set:

In [ ]:
predictions_luna = model.predict(np.array(testx_luna)[:,:,:,np.newaxis]).squeeze()
gt_luna = np.array(testy_luna)
gtx_luna=np.array(testx_luna)

In [ ]:
# get total lung volume values
scale_factor = 512/256 
mag = (1800-120)/1800 #from  dicom data, standard for most ct scanners
gt_vol_luna = [gt_luna[i].sum() * (pixel_spacings_luna[i][0]*scale_factor*mag)**2 *1e-6 for i in range(len(gt_luna))]
pred_vol_luna = [predictions_luna[i].sum() * (pixel_spacings_luna[i][0]*scale_factor*mag)**2 *1e-6 for i in range(len(predictions_luna))]

In [ ]:
# Check that data is from a normal distribution before getting 95% CI with all 3 test
shapiro_wilk(gt_vol_luna), shapiro_wilk(pred_vol_luna), dAgostino_pearson(gt_vol_luna), dAgostino_pearson(pred_vol_luna),anderson_darling(gt_vol_luna), anderson_darling(pred_vol_luna)

In [ ]:
# get metrics
corr_luna, pvalue_luna = pearsonr(gt_vol_luna,pred_vol_luna)
print("mean vol_gt = {:.3f}, mean vol_pred = {:.3f}".format(np.mean(gt_vol_luna), np.mean(pred_vol_luna)))
print('95% CI for mean vol_gt = {:.3f}, {:.3f}, mean vol_pred = {:.3f}, {:.3f}'.format(get_metric(gt_vol_luna, '95ci')[0], get_metric(gt_vol_luna, '95ci')[1], get_metric(pred_vol_luna, '95ci')[0], get_metric(pred_vol_luna, '95ci')[1]))
print("Pearson corr = {:.3f},  P-value = {}, n = {}".format(corr_luna, pvalue_luna,len(gt_vol_luna) ))
print("MSE:  %.3f, MAE = %.3f, MAPE = %.3f"%(mse(gt_vol_luna, pred_vol_luna), mae(gt_vol_luna, pred_vol_luna),  mape(gt_vol_luna, pred_vol_luna)))

In [ ]:
plotter(gtx_luna, gt_luna, predictions_luna, name='_Luna16Test_finalSegs', save=False, y_idx=np.linspace(44,56,11)) #y_idx=[0,1,2,3,4,12,15,7,16,9,11,11]

### 2b) Predict on PE test set:

In [ ]:
predictions_PE = model.predict(np.array(testx_PE)[:,:,:,np.newaxis]).squeeze()
gt_PE = np.array(testy_PE)
gtx_PE=np.array(testx_PE)

In [ ]:
# check to see if any of the gt volumes are zero, exclude these patients
[i for i in range(len(gt_PE)) if np.isclose(gt_PE[i].sum(),0)]

In [ ]:
# get total lung volume values:
scale_factor = 512/256
gt_vol_PE = [gt_PE[i].sum()*pixel_spacings_PE[i][0]*pixel_spacings_PE[i][2]*scale_factor_pre_PE[i]*(scale_factor*mag)**2 *1e-6 for i in range(len(gt_PE)) if not np.isclose(gt_PE[i].sum(), 0)]#i not in [78, 323, 446, 539] ]
pred_vol_PE = [predictions_PE[i].sum()*pixel_spacings_PE[i][0]*pixel_spacings_PE[i][2]*scale_factor_pre_PE[i]*(scale_factor*mag)**2 *1e-6 for i in range(len(predictions_PE)) if not np.isclose(gt_PE[i].sum(), 0)]

In [ ]:
# Check that data is from a normal distribution before getting 95% CI with all 3 tests:
shapiro_wilk(gt_vol_PE), shapiro_wilk(pred_vol_PE), dAgostino_pearson(gt_vol_PE), dAgostino_pearson(pred_vol_PE), anderson_darling(gt_vol_PE), anderson_darling(pred_vol_PE)

In [ ]:
# get metrics
corr_PE, pvalue_PE = pearsonr(gt_vol_PE,pred_vol_PE)
print("mean vol_gt = %.3f, mean vol_pred = %.3f"% (np.mean(gt_vol_PE), np.mean(pred_vol_PE)))
print('95% CI for mean vol_gt = {:.3f}, {}, mean vol_pred = {:.3f}, {:.3f}'.format(get_metric(gt_vol_PE, '95ci')[0], get_metric(gt_vol_PE, '95ci')[1], get_metric(pred_vol_PE, '95ci')[0], get_metric(pred_vol_PE, '95ci')[1]))
print("Pearson corr = %.3f,  P-value = %.3f, n = %i"% (corr_PE,pvalue_PE, len(gt_vol_PE) ))
print("MSE:  %.3f, MAE = %.3f, MAPE = %.3f"%(mse(gt_vol_PE, pred_vol_PE), mae(gt_vol_PE, pred_vol_PE),  mape(gt_vol_PE, pred_vol_PE)))

In [ ]:
# processes (stretch and clip) PE data so that it looks right when plotted
gtx_PE_processed = np.array([process_PE(i, gtx_PE, pixel_spacings_PE, slices_z) for i in range(0,200)])
gt_PE_processed = np.array([process_PE(i, gt_PE, pixel_spacings_PE, slices_z) for i in range(0,200)])
predictions_PE_processed = np.array([process_PE(i, predictions_PE, pixel_spacings_PE, slices_z) for i in range(0,200)])
plotter(gtx_PE_processed, gt_PE_processed, predictions_PE_processed, y_idx=[0, 166, 22, 38, 39, 44, 56, 166, 121, 143, 149], name='PeTest_finalSegProcessed', save=False) 

### 2c) Predict on both PE and Luna16 test sets combined:

In [ ]:
n_all = len(np.concatenate([pred_vol_luna, pred_vol_PE]))
corr_all_ct, pvalue_all_ct = pearsonr(np.concatenate([pred_vol_luna, pred_vol_PE]),np.concatenate([gt_vol_luna, gt_vol_PE]))
print("Synthetic: Pearson corr = {},  P-value = {}, n = {}".format(corr_all_ct,pvalue_all_ct, n_all ))


In [ ]:
## combined scatter plots:
plt.figure(figsize=(10,4))
plt.subplot(121)
coef_luna_ct = np.polyfit(pred_vol_luna, gt_vol_luna,1)
poly1d_fn = np.poly1d(coef_luna_ct) 

coef_PE_ct = np.polyfit(pred_vol_PE, gt_vol_PE,1)
poly1d_fn = np.poly1d(coef_PE_ct) 

coef = np.polyfit(np.concatenate([pred_vol_luna, pred_vol_PE]) , np.concatenate([gt_vol_luna, gt_vol_PE]),1)
poly1d_fn = np.poly1d(coef) 
plt.plot(np.concatenate([pred_vol_luna, pred_vol_PE]), poly1d_fn(np.concatenate([pred_vol_luna, pred_vol_PE])), 'k', label='f(x) = mx+b ; m={:.2f}, b={:.3f}'.format(coef[0], coef[1]), lw=1)

plt.scatter([pred_vol_PE[i] for i in range(len(gt_vol_PE)) if gt_vol_PE[i] < 11.7], [gt_vol_PE[i] for i in range(len(gt_vol_PE)) if gt_vol_PE[i] < 11.7], label='PE', alpha=0.5)
plt.scatter([pred_vol_luna[i] for i in range(len(gt_vol_luna)) if gt_vol_luna[i] <11.7] , [gt_vol_luna[i] for i in range(len(gt_vol_luna)) if gt_vol_luna[i] <11.7], label='Luna16', alpha=0.6)
plt.title('Synthetic (n = {}): r = {:.3f} (P < 0.001)'.format(n_all, corr_all_ct))

plt.legend(loc='upper left')
plt.plot(range(0,17), range(0,17), 'k', alpha=0.2, lw=0.5)
plt.ylabel('Ground Truth Volume [L] ')
plt.xlabel('Predicted Volume [L] ')
plt.xlim(0,11.99)
plt.ylim(0,11.99)
plt.grid(alpha=0.2)
plt.subplot(122)
plt.tight_layout(pad=1.0, w_pad=1, h_pad=1)
